In [ ]:
# import os
# import shutil
# from PIL import Image
# from paddleocr import PaddleOCR
# from tqdm import tqdm
# from datetime import datetime
# import math
# import time
# import numpy as np
# import pandas as pd

# # 定义基础路径
# base_path = 'D://code//data//Lv2期结论//女鞋_from_0501'

# # 初始化 PaddleOCR
# ocr = PaddleOCR(use_angle_cls=True, lang="ch", show_log=False)

# def move_images_with_white_pixels(source_folder, white_folder, threshold=0.40):
#     image_files = [f for f in os.listdir(source_folder) if f.endswith(('.jpg', '.png'))]
    
#     with tqdm(total=len(image_files), desc="Processing images") as pbar:
#         for filename in image_files:
#             img_path = os.path.join(source_folder, filename)
#             img = Image.open(img_path)
#             img_np = np.array(img)

#             # 如果图片中有文本，则不移动到 white_folder
#             result = ocr.ocr(img_np, cls=True)
#             if result is not None and result[0] is not None:
#                 ignored_keywords = ['京喜自营', '包邮']
#                 filtered_data_list = [data for data in result[0] if all(keyword not in data[1][0] for keyword in ignored_keywords)]
#                 if filtered_data_list:
#                     pbar.update(1)
#                     continue

#             # 计算图片中白色像素的比例
#             white_pixels = np.sum(np.all(img_np >= [240, 240, 240], axis=2))
#             total_pixels = img_np.shape[0] * img_np.shape[1]
#             white_pixels_ratio = white_pixels / total_pixels

#             if white_pixels_ratio >= threshold:
#                 try:
#                     shutil.move(img_path, white_folder)
#                     pbar.write(f"Image '{filename}' moved to white_folder.")
#                 except Exception as e:
#                     pbar.write(f"Failed to move image '{filename}': {e}")
            
#             pbar.update(1)


# def process_subfolder(subfolder_path):
#     grounding_output_path = os.path.join(subfolder_path, 'grounding_output')
#     if not os.path.exists(grounding_output_path):
#         # print(f"No grounding_output folder in {subfolder_path}")
#         return

#     # 创建分类文件夹
#     price_folder = os.path.join(grounding_output_path, 'price')
#     txt_folder = os.path.join(grounding_output_path, 'txt')
#     scene_folder = os.path.join(grounding_output_path, 'scene')
#     white_folder = os.path.join(grounding_output_path, 'white')

#     for folder in [price_folder, txt_folder, scene_folder, white_folder]:
#         if not os.path.exists(folder):
#             os.makedirs(folder)

#     # 获取所有图片文件
#     image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

#     # Step 1: 筛选价促卖点图
#     print('Step 1: 筛选价促卖点图')
#     with tqdm(total=len(image_files), desc="Processing images") as pbar:
#         for filename in image_files:
#             img_path = os.path.join(grounding_output_path, filename)
            
#             result = ocr.ocr(img_path, cls=True)
            
#             if not result:
#                 pbar.update(1)
#                 continue
            
#             data_list = result[0]
            
#             keywords = ['满', '减', '到手价', '送', '免息', '活动价', '包邮价', '参考价', 
#                         r'.*满.*减.*', r'.*满.*-.*', r'.*满.*赠.*',r'.*满.*送.*', r'.*价.*', 
#                         '券', '优惠', '用券', '领券', '券', '送', '低至', '立减', '直降', '免息',
#                         '¥', '夫', '￥','分钱']
            
#             # 忽略“京喜自营”和“包邮”
#             ignored_keywords = ['京喜自营', '包邮']
#             filtered_data_list = [data for data in data_list if all(keyword not in data[1][0] for keyword in ignored_keywords)]
            
#             contains_keyword = any(any(keyword in data[1][0] for keyword in keywords) for data in filtered_data_list)
            
#             if contains_keyword:
#                 try:
#                     shutil.move(img_path, price_folder)
#                     pbar.write(f"Image '{filename}' moved to price_folder.")
#                 except Exception as e:
#                     pbar.write(f"Failed to move image '{filename}': {e}")
            
#             pbar.update(1)

#     # 更新图片文件列表，排除已移动的文件
#     image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

#     # Step 2: 筛选白底图
#     print('Step 2: 筛选白底图')
#     move_images_with_white_pixels(grounding_output_path, white_folder, threshold=0.40)

#     # 更新图片文件列表，排除已移动的文件
#     image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

#     # Step 3: 筛选功能卖点图
#     print('Step 3: 筛选功能卖点图')
#     with tqdm(total=len(image_files), desc="Processing images") as pbar:
#         for filename in image_files:
#             img_path = os.path.join(grounding_output_path, filename)
            
#             img = Image.open(img_path)
#             width, height = img.size
            
#             cropped_img = img.crop((0, height // 6, width, height))
#             img_np = np.array(cropped_img)
            
#             result = ocr.ocr(img_np, cls=True)
            
#             if result is None or result[0] is None:
#                 pbar.update(1)
#                 continue
                
#             # 忽略“京喜自营”和“包邮”
#             ignored_keywords = ['京喜自营', '包邮']
#             filtered_data_list = [data for data in result[0] if any(keyword not in data[1][0] for keyword in ignored_keywords)]
            
#             rectangles_with_text = filtered_data_list
#             line_count = len(rectangles_with_text)
            
#             if line_count > 0:
#                 shutil.move(img_path, txt_folder)
#                 # print(f"'{filename}' moved to txt folder.")
#             else:
#                 # print(f"'{filename}' remains in source folder.")
#                 pbar.update(1)

#     # Step 4: 将剩余图片归类到scene
#     print('Step 4: 归类图片到scene')
#     image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]
#     for filename in image_files:
#         img_path = os.path.join(grounding_output_path, filename)
#         shutil.move(img_path, scene_folder)
#         # print(f"Image '{filename}' moved to scene_folder.")

#     # Step 5: 将white中有文本的图片移到txt_folder
#     print('Step 5: 将white中有文本的图片移到txt_folder')
#     image_files = [f for f in os.listdir(white_folder) if f.endswith(('.jpg', '.png'))]
#     with tqdm(total=len(image_files), desc="Processing images") as pbar:
#         for filename in image_files:
#             img_path = os.path.join(white_folder, filename)
#             img = Image.open(img_path)
#             width, height = img.size
#             cropped_img = img.crop((0, height // 6, width, height))
#             img_np = np.array(cropped_img)
#             result = ocr.ocr(img_np, cls=True)
            
#             if result is None or result[0] is None:
#                 pbar.update(1)
#                 continue
                
#             # 忽略“京喜自营”和“包邮”
#             ignored_keywords = ['京喜自营', '包邮']
#             filtered_data_list = [data for data in result[0] if all(keyword not in data[1][0] for keyword in ignored_keywords)]
            
#             rectangles_with_text = filtered_data_list
#             lines_above_30_count = sum(1 for rectangle in rectangles_with_text if min(math.sqrt((x - rectangle[0][0][0])**2 + (y - rectangle[0][0][1])**2) for x, y in rectangle[0][1:]) > 30)
                
#             if lines_above_30_count >= 2:
#                 shutil.move(img_path, txt_folder)
#                 pbar.write(f"Image '{filename}' moved to txt_folder.")
#             else:
#                 pbar.write(f"Image '{filename}' remains in white folder.")
            
#             pbar.update(1)

# # 主程序
# subfolders = [f.path for f in os.scandir(base_path) if f.is_dir()]

# for subfolder in subfolders:
#     print(f"Processing subfolder: {subfolder}")
#     process_subfolder(subfolder)

# print('All processing completed.')

# # 获取当前时间
# current_time = datetime.now()
# print("完成时间:", current_time)





In [9]:
import os
import shutil
from PIL import Image
from paddleocr import PaddleOCR
from tqdm import tqdm
from datetime import datetime
import math
import time
import numpy as np
import pandas as pd


%config IPCompleter.greedy=True


def process_folder(root_folder):
    # 初始化 PaddleOCR
    ocr = PaddleOCR(use_angle_cls=True, lang="ch", show_log=False)

    # 遍历根文件夹下的所有子文件夹
    for dirpath, dirnames, filenames in os.walk(root_folder):
        if os.path.basename(dirpath) == 'grounding_output':
            price_folder = os.path.join(dirpath, 'price')
            txt_folder = os.path.join(dirpath, 'txt')
            scene_folder = os.path.join(dirpath, 'scene')
            white_folder = os.path.join(dirpath, 'white')

            # 如果文件夹不存在，则创建
            for folder in [price_folder, txt_folder, scene_folder, white_folder]:
                if not os.path.exists(folder):
                    os.makedirs(folder)

            '''
            step1 - 筛选价促卖点图
            '''
            print('Step 1: 筛选价促卖点图')

            image_files = [os.path.join(dirpath, filename) for filename in os.listdir(dirpath) if filename.endswith(('.jpg', '.png'))]

            # 使用 tqdm 创建进度条
            with tqdm(total=len(image_files), desc="Processing images") as pbar:
                # 遍历源文件夹中的所有图片文件
                for filename in image_files:
                    img_path = os.path.join(dirpath, filename)

                    # 使用 PaddleOCR 进行文字识别
                    result = ocr.ocr(img_path, cls=True)

                    if not result:
                        # 如果识别结果为空,则跳过这张图片,不进行移动操作
                        pbar.write(f"Image '{filename}' skipped due to empty OCR result.")
                        pbar.update(1)
                        continue

                    data_list = result[0]

                    # 检查识别结果是否有关键词
                    contains_keyword = False

                    if data_list:
                        # 定义关键词列表
                        keywords = ['满', '减', '折', '到手价', '送', '免息', '活动价', '包邮价', '参考价',
                                    r'.*满.*减.*', r'.*满.*-.*', r'.*满.*赠.*', r'.*满.*送.*', r'.*价.*',
                                    '券', '优惠', '用券', '领券', '券', '送', '低至', '立减', '直降', '免息',
                                    '¥', '夫', '￥']

                        # 遍历识别结果中的文本
                        for data in data_list:
                            text = data[1][0]  # 获取文本内容
                            # 检查当前文本是否包含关键词
                            if any(keyword in text for keyword in keywords):
                                contains_keyword = True
                                break

                    # 根据检查结果移动文件
                    if contains_keyword:
                        try:
                            shutil.move(img_path, price_folder)
                            # pbar.write(f"Image '{filename}' moved to price_folder.")
                        except Exception as e:
                            pbar.write(f"Failed to move image '{filename}': {e}")
                            continue
                    # else:
                    #     # 如果不包含关键词，移动到其他文件夹
                    #     # 你可以根据需要修改这里的逻辑
                    #     try:
                    #         shutil.move(img_path, txt_folder)
                    #         pbar.write(f"Image '{filename}' moved to scene_folder.")
                    #     except Exception as e:
                    #         pbar.write(f"Failed to move image '{filename}': {e}")

                    # 重置 contains_keyword 变量
                    contains_keyword = False

                    # 更新进度条
                    pbar.update(1)

            print("卖点图 classification completed.")

            # 获取当前时间
            current_time = datetime.now()

            # 格式化输出当前时间
            print("完成时间:", current_time)

            '''
            step2 - 筛选白底图
            '''

            print('Step 2: 筛选白底图')

            def move_images_with_white_pixels(base_path, white_folder, threshold=0.40):
                # 获取源文件夹中的图片文件列表
                image_files = [filename for filename in os.listdir(base_path) if filename.endswith(('.jpg', '.png'))]

                # 使用 tqdm 创建进度条
                with tqdm(total=len(image_files), desc=f"Moving images from {base_path}") as pbar:
                    # 遍历源文件夹中的所有图片文件
                    for filename in image_files:
                        img_path = os.path.join(base_path, filename)

                        # 打开图片并获取像素信息
                        with Image.open(img_path) as img:
                            # 确认图片是 RGB 模式
                            if img.mode!= 'RGB':
                                img = img.convert('RGB')
                            # 获取图片的宽度和高度
                            width, height = img.size

                            # 统计白色像素点的数量
                            white_pixels = 0
                            for x in range(width):
                                for y in range(height):
                                    # 获取像素点的 RGB 值
                                    pixel_value = img.getpixel((x, y))
                                    # 如果是 RGB 图像，解包为三个值，否则为四个值
                                    if len(pixel_value) == 3:
                                        r, g, b = pixel_value
                                    else:  # 处理带有透明度的图像
                                        r, g, b, a = pixel_value
                                    # 如果 RGB 值都大于 230，则认为是白色像素点
                                    if r > 230 and g > 230 and b > 230:
                                        white_pixels += 1

                            # 计算白色像素点占比
                            white_ratio = white_pixels / (width * height)

                            # 如果白色像素点占比超过阈值，则将图片移动到目标文件夹
                            if white_ratio > threshold:
                                target_path = os.path.join(white_folder, filename)
                                try:
                                    shutil.move(img_path, white_folder)
                                    # pbar.write(f"Image '{filename}' moved to white folder.")
                                except Exception as e:
                                    pbar.write(f"Error moving image '{filename}': {e}")
                                    continue

                        # 更新进度条
                        pbar.update(1)

            # 调用函数，将白色像素点占比超过 30%的图片从源文件夹列表中移动到目标文件夹
            move_images_with_white_pixels(dirpath, white_folder, threshold=0.40)

            # 获取当前时间
            current_time = datetime.now()

            # 格式化输出当前时间
            print("完成时间:", current_time)

            '''
            step3 - 筛选功能卖点图
            '''

            print('Step 3: 筛选功能卖点图')

            image_files = [filename for filename in os.listdir(dirpath) if filename.endswith(('.jpg', '.png'))]
            for filename in image_files:

                img_path = os.path.join(dirpath, filename)
                # print(f"Processing image: {img_path}")

                img = Image.open(img_path)
                width, height = img.size

                # 读取图片下部分 5/6
                cropped_img = img.crop((0, height // 6, width, height))

                # 将 PIL 图像对象转换为 numpy 数组
                img_np = np.array(cropped_img)

                # OCR 处理
                result = ocr.ocr(img_np, cls=True)

                if result is None:
                    continue

                rectangles_with_text = result[0]

                if rectangles_with_text is None:
                    continue

                line_count = len(rectangles_with_text)

                # 如果文本行数大于等于 3, 将图片移动到目标文件夹
                if line_count >= 2:
                    target_path = os.path.join(txt_folder, filename)
                    shutil.move(img_path, txt_folder)
                    # print(f"'{filename}' moved to txt folder.")

                else:
                    # print(f"'{filename}' remains in source folder.")
                    continue

            '''
            step4 - 将剩余图片归类到 scene
            '''

            print('Step 4: 归类图片到 scene')

            # 检查目标文件夹是否存在，如果不存在则创建
            if not os.path.exists(scene_folder):
                os.makedirs(scene_folder)

            # 获取源文件夹中的所有图片文件列表
            image_files = [filename for filename in os.listdir(dirpath) if filename.endswith(('.jpg', '.png'))]

            # 移动图片到 txt_folder
            for filename in image_files:
                img_path = os.path.join(dirpath, filename)
                target_path = os.path.join(scene_folder, filename)
                shutil.move(img_path, scene_folder)
                # print(f"Image '{filename}' moved to scene_folder.")

            print("Image moving completed.")

            # 获取当前时间
            current_time = datetime.now()

            # 格式化输出当前时间
            print("完成时间:", current_time)

            '''
            step5 - 将 white 中有文本的图片移到 txt_folder
            '''

            # 获取源文件夹中的所有图片文件列表
            image_files = [filename for filename in os.listdir(white_folder) if filename.endswith(('.jpg', '.png'))]

            # 使用 tqdm 创建进度条
            with tqdm(total=len(image_files), desc="Processing images") as pbar:

                # 遍历源文件夹中的所有图片文件
                for filename in image_files:
                    img_path = os.path.join(white_folder, filename)

                    img = Image.open(img_path)
                    width, height = img.size

                    # 读取图片下部分 3/4
                    cropped_img = img.crop((0, height // 6, width, height))

                    # 将 PIL 图像对象转换为 numpy 数组
                    img_np = np.array(cropped_img)

                    # OCR 处理
                    result = ocr.ocr(img_np, cls=True)

                    if result is None:
                        continue

                    rectangles_with_text = result[0]

                    if rectangles_with_text is None:
                        continue

                    # 统计文本框高度大于 30 像素的行数
                    lines_above_30_count = 0
                    for rectangle in rectangles_with_text:
                        # 计算文本框的高度和宽度
                        points = rectangle[0]
                        x_A, y_A = points[0]

                        # 计算 A 点到 BCD 的距离
                        distances = []
                        for point in points[1:]:
                            x_B, y_B = point
                            distance = math.sqrt((x_B - x_A) ** 2 + (y_B - y_A) ** 2)
                            distances.append(distance)

                        # 获取最短的距离作为文本框的大小
                        text_size = min(distances)

                        # 统计高度大于 x 像素的行数
                        if text_size > 30:
                            lines_above_30_count += 1

                    # 如果大于 30 像素的行数大于等于 2，将图片移动到目标文件夹
                    if lines_above_30_count >= 2:
                        # 如果识别结果不为空且行数小于等于 3，则将图片复制到 txt_folder
                        target_path = os.path.join(txt_folder, filename)
                        shutil.move(img_path, target_path)
                        # pbar.write(f"Image '{filename}' copied to txt_folder.")
                    else:
                        # pbar.write(f"Image '{filename}' remains in source folder.")
                        continue

                    # 更新进度条
                    pbar.update(1)

            print("Image classification completed.")

            print('okkk')

            # 获取当前时间
            current_time = datetime.now()

            # 打印当前时间
            print("当前系统时间是:", current_time)
            print("当前系统时间是:", current_time)
            print("当前系统时间是:", current_time)


root_folder = "D://code//data//Lv2期结论//男鞋_from_0501"
process_folder(root_folder)




Step 1: 筛选价促卖点图


Processing images: 100%|██████████| 3/3 [00:01<00:00,  2.06it/s]


卖点图 classification completed.
完成时间: 2024-10-25 10:44:21.967672
Step 2: 筛选白底图


Moving images from D://code//data//Lv2期结论//男鞋_from_0501\12066\grounding_output: 100%|██████████| 3/3 [00:00<00:00,  3.93it/s]


完成时间: 2024-10-25 10:44:22.743692
Step 3: 筛选功能卖点图
Step 4: 归类图片到 scene
Image moving completed.
完成时间: 2024-10-25 10:44:23.433699


...

卖点图 classification completed.
完成时间: 2024-10-25 15:04:43.964532
Step 2: 筛选白底图


Moving images from D://code//data//Lv2期结论//男鞋_from_0501\9783\grounding_output: 100%|██████████| 7/7 [00:01<00:00,  3.73it/s]


完成时间: 2024-10-25 15:04:45.905748
Step 3: 筛选功能卖点图
Step 4: 归类图片到 scene
Image moving completed.
完成时间: 2024-10-25 15:04:49.633090


Processing images:   0%|          | 0/2 [00:02<?, ?it/s]


Image classification completed.
okkk
当前系统时间是: 2024-10-25 15:04:52.229769
当前系统时间是: 2024-10-25 15:04:52.229769
当前系统时间是: 2024-10-25 15:04:52.229769


# step2 - 对四分类的数据总结

In [1]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import openpyxl
from openpyxl.drawing.image import Image as XLImage


z = '男鞋_from_0501'


# 定义基础路径和CSV文件路径
base_path = f'D://code//data//Lv2期结论//{z}'
csv_file_path = f'D://code//data//Lv2期结论//{z}//{z}.csv'


# 读取CSV文件
df = pd.read_csv(csv_file_path)


def extract_matching_part(img_url):
    if pd.isna(img_url):
        return None
    img_url = img_url.split('?')[0]
    img_url = os.path.splitext(img_url)[0]
    parts = img_url.split('/')
    if len(parts) >= 2:
        return f"{parts[-2]}_{parts[-1]}"
    return None

df['matching_part'] = df['img_url'].apply(extract_matching_part)

def process_grounding_folder(grounding_path):
    folder_stats = {
        'price': {'count': 0, 'uv': 0, 'click_uv': 0},
        'txt': {'count': 0, 'uv': 0, 'click_uv': 0},
        'scene': {'count': 0, 'uv': 0, 'click_uv': 0},
        'white': {'count': 0, 'uv': 0, 'click_uv': 0}
    }

    for folder_name in ['price', 'txt', 'scene', 'white']:
        folder_path = os.path.join(grounding_path, folder_name)
        if os.path.exists(folder_path):
            for filename in os.listdir(folder_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    filename_without_ext = os.path.splitext(filename)[0]
                    filtered_df = df[df['matching_part'] == filename_without_ext]
                    if not filtered_df.empty:
                        folder_stats[folder_name]['count'] += 1
                        folder_stats[folder_name]['uv'] += filtered_df['uv'].sum()
                        folder_stats[folder_name]['click_uv'] += filtered_df['click_uv'].sum()

    # 计算每个文件夹的CTR
    for folder in folder_stats:
        if folder_stats[folder]['uv'] > 0:
            folder_stats[folder]['ctr'] = folder_stats[folder]['click_uv'] / folder_stats[folder]['uv']
        else:
            folder_stats[folder]['ctr'] = 0

    return folder_stats

all_stats = {}

for root, dirs, files in os.walk(base_path):
    if 'grounding_output' in dirs:
        grounding_path = os.path.join(root, 'grounding_output')
        folder_name = os.path.basename(root)
        stats = process_grounding_folder(grounding_path)
        all_stats[folder_name] = stats

# 创建一个Excel工作簿来保存结果
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.title = "Image Distribution Stats"

# 写入表头
headers = ["Folder", "Subfolder", "Count", "UV", "Click UV", "CTR"]
for col, header in enumerate(headers, start=1):
    sheet.cell(row=1, column=col, value=header)

row = 2  # 从第二行开始写入数据

# 输出统计结果
for folder, stats in all_stats.items():
    print(f"\n统计结果 for {folder}:")
    for subfolder, data in stats.items():
        if data['count'] > 0:
            ctr = data['click_uv'] / data['uv'] if data['uv'] > 0 else 0
            print(f"  {subfolder}: 图片数量 = {data['count']}, UV = {data['uv']}, Click UV = {data['click_uv']}, CTR = {ctr:.4f}")


# 定义颜色映射
color_map = {
    'price': '#FF6B6B',  # 柔和的红色
    'txt': '#4ECDC4',    # 青绿色
    'scene': '#7986CB',  # 淡紫色
    'white': '#FFD93D'   # 明亮的黄色
}



# 绘制每个grounding_output文件夹的饼图并保存数据
for folder, stats in all_stats.items():
    counts = [data['count'] for data in stats.values() if data['count'] > 0]
    labels = [subfolder for subfolder, data in stats.items() if data['count'] > 0]
    ctrs = [data['ctr'] for data in stats.values() if data['count'] > 0]
    colors = [color_map.get(label, 'gray') for label in labels]  # 使用颜色映射，如果没有指定则默认为灰色
    
    if counts:  # 只有当有数据时才绘图
        plt.figure(figsize=(12, 9))
        wedges, texts, autotexts = plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
        
        # 添加数量和CTR标签
        for i, (autotext, ctr) in enumerate(zip(autotexts, ctrs)):
            autotext.set_text(f'{autotext.get_text()}\n({counts[i]})\nCTR: {ctr:.4f}')
        
        plt.title(f"Image Distribution in {folder}")
        plt.axis('equal')
        
        # 添加图例
        plt.legend(wedges, [f"{label} ({count}, CTR: {ctr:.4f})" for label, count, ctr in zip(labels, counts, ctrs)],
                   title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
        
        output_chart_path = os.path.join(base_path, f'{folder}_distribution.png')
        plt.savefig(output_chart_path, bbox_inches='tight')
        plt.close()
        print(f"饼图已保存至: {output_chart_path}")

        # 将图片插入到Excel中
        img = XLImage(output_chart_path)
        sheet.add_image(img, f'H{row}')

        # 写入数据到Excel
        for subfolder, data in stats.items():
            if data['count'] > 0:
                sheet.cell(row=row, column=1, value=folder)
                sheet.cell(row=row, column=2, value=subfolder)
                sheet.cell(row=row, column=3, value=data['count'])
                sheet.cell(row=row, column=4, value=data['uv'])
                sheet.cell(row=row, column=5, value=data['click_uv'])
                sheet.cell(row=row, column=6, value=data['ctr'])
                row += 1

        row += 15  # 为下一个文件夹的数据和图表留出空间

# 计算总体统计
total_stats = {
    'price': {'count': 0, 'uv': 0, 'click_uv': 0},
    'txt': {'count': 0, 'uv': 0, 'click_uv': 0},
    'scene': {'count': 0, 'uv': 0, 'click_uv': 0},
    'white': {'count': 0, 'uv': 0, 'click_uv': 0}
}

for folder_stats in all_stats.values():
    for subfolder, data in folder_stats.items():
        total_stats[subfolder]['count'] += data['count']
        total_stats[subfolder]['uv'] += data['uv']
        total_stats[subfolder]['click_uv'] += data['click_uv']

# 计算总体CTR
for subfolder in total_stats:
    if total_stats[subfolder]['uv'] > 0:
        total_stats[subfolder]['ctr'] = total_stats[subfolder]['click_uv'] / total_stats[subfolder]['uv']
    else:
        total_stats[subfolder]['ctr'] = 0

# 打印总体统计结果
print("\n总体统计结果:")
for subfolder, data in total_stats.items():
    if data['count'] > 0:
        print(f"  {subfolder}: 总图片数量 = {data['count']}, 总UV = {data['uv']}, 总Click UV = {data['click_uv']}, 总CTR = {data['ctr']:.4f}")

# 绘制总体分布饼图
total_counts = [data['count'] for data in total_stats.values() if data['count'] > 0]
total_labels = [subfolder for subfolder, data in total_stats.items() if data['count'] > 0]
total_ctrs = [data['click_uv'] / data['uv'] if data['uv'] > 0 else 0 for data in total_stats.values() if data['count'] > 0]
total_colors = [color_map.get(label, 'gray') for label in total_labels]

if total_counts:
    plt.figure(figsize=(12, 9))
    wedges, texts, autotexts = plt.pie(total_counts, labels=total_labels, autopct='%1.1f%%', startangle=90, colors=total_colors)
    
    # 添加数量和CTR标签
    for i, (autotext, ctr) in enumerate(zip(autotexts, total_ctrs)):
        autotext.set_text(f'{autotext.get_text()}\n({total_counts[i]})\nCTR: {ctr:.4f}')
    
    plt.title("Overall Image Distribution")
    plt.axis('equal')
    
    # 添加图例
    plt.legend(wedges, [f"{label} ({count}, CTR: {ctr:.4f})" for label, count, ctr in zip(total_labels, total_counts, total_ctrs)],
               title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    
    output_chart_path = os.path.join(base_path, 'overall_distribution.png')
    plt.savefig(output_chart_path, bbox_inches='tight')
    plt.close()
    print(f"总体分布饼图已保存至: {output_chart_path}")

    # 将总体分布图片插入到Excel中
    img = XLImage(output_chart_path)
    sheet.add_image(img, f'H{row}')

    # 写入总体数据到Excel
    sheet.cell(row=row, column=1, value="Overall")
    for subfolder, data in total_stats.items():
        if data['count'] > 0:
            sheet.cell(row=row, column=2, value=subfolder)
            sheet.cell(row=row, column=3, value=data['count'])
            sheet.cell(row=row, column=4, value=data['uv'])
            sheet.cell(row=row, column=5, value=data['click_uv'])
            sheet.cell(row=row, column=6, value=data['click_uv'] / data['uv'] if data['uv'] > 0 else 0)
            row += 1

# 保存Excel文件
excel_output_path = os.path.join(base_path, 'image_distribution_stats.xlsx')
workbook.save(excel_output_path)
print(f"统计结果已保存至Excel文件: {excel_output_path}")


    


统计结果 for 12066:
  scene: 图片数量 = 3, UV = 119, Click UV = 9, CTR = 0.0756

统计结果 for 6908:
  price: 图片数量 = 462, UV = 1997399, Click UV = 75119, CTR = 0.0376
  txt: 图片数量 = 120, UV = 2777440, Click UV = 90451, CTR = 0.0326
  scene: 图片数量 = 721, UV = 14113428, Click UV = 461185, CTR = 0.0327
  white: 图片数量 = 246, UV = 2132073, Click UV = 76381, CTR = 0.0358

统计结果 for 6909:
  price: 图片数量 = 446, UV = 787151, Click UV = 28230, CTR = 0.0359
  txt: 图片数量 = 275, UV = 2535197, Click UV = 100304, CTR = 0.0396
  scene: 图片数量 = 556, UV = 6398186, Click UV = 275823, CTR = 0.0431
  white: 图片数量 = 263, UV = 1562568, Click UV = 60597, CTR = 0.0388

统计结果 for 6910:
  price: 图片数量 = 63, UV = 50103, Click UV = 2934, CTR = 0.0586
  txt: 图片数量 = 169, UV = 844799, Click UV = 27781, CTR = 0.0329
  scene: 图片数量 = 242, UV = 1738848, Click UV = 74195, CTR = 0.0427
  white: 图片数量 = 261, UV = 497952, Click UV = 20120, CTR = 0.0404

统计结果 for 6911:
  price: 图片数量 = 50, UV = 1135608, Click UV = 54777, CTR = 0.0482
  txt: 图片数量 = 2

In [5]:
# 添加了品牌分层的统计逻辑
# 添加了品牌分层的统计逻辑
# 添加了品牌分层的统计逻辑



import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import openpyxl
from openpyxl.drawing.image import Image as XLImage



# z = '女士春夏下装_from_0501'

# 定义基础路径和CSV文件路径
base_path = f'D://code//data//Lv2期结论//{z}'
csv_file_path = f'D://code//data//Lv2期结论//{z}//{z}.csv'
brand_path = f'D://code//data//Lv2期结论//{z}//男鞋品牌分层.xlsx'  # 品牌分类数据源



# 读取CSV文件
df = pd.read_csv(csv_file_path)
df_brand = pd.read_excel(brand_path)  # 读取品牌分类信息

# 合并df和df_brand数据
df = pd.merge(df, df_brand, on='main_brand_code', how='left')

# 定义筛选条件
filter_layers = [5.0, 6.0]  # 替换为你想要的筛选条件，可以是多个值
# filter_layers = [1.0, 2.0, 3.0]  # 替换为你想要的筛选条件，可以是多个值



# 筛选数据
filtered_df = df[df['最终分层'].isin(filter_layers)]

def extract_matching_part(img_url):
    if pd.isna(img_url):
        return None
    img_url = img_url.split('?')[0]
    img_url = os.path.splitext(img_url)[0]
    parts = img_url.split('/')
    if len(parts) >= 2:
        return f"{parts[-2]}_{parts[-1]}"
    return None

filtered_df['matching_part'] = filtered_df['img_url'].apply(extract_matching_part)

def process_grounding_folder(grounding_path):
    folder_stats = {
        'price': {'count': 0, 'uv': 0, 'click_uv': 0},
        'txt': {'count': 0, 'uv': 0, 'click_uv': 0},
        'scene': {'count': 0, 'uv': 0, 'click_uv': 0},
        'white': {'count': 0, 'uv': 0, 'click_uv': 0}
    }

    for folder_name in ['price', 'txt', 'scene', 'white']:
        folder_path = os.path.join(grounding_path, folder_name)
        if os.path.exists(folder_path):
            for filename in os.listdir(folder_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    filename_without_ext = os.path.splitext(filename)[0]
                    folder_filtered_df = filtered_df[filtered_df['matching_part'] == filename_without_ext]
                    if not folder_filtered_df.empty:
                        folder_stats[folder_name]['count'] += 1
                        folder_stats[folder_name]['uv'] += folder_filtered_df['uv'].sum()
                        folder_stats[folder_name]['click_uv'] += folder_filtered_df['click_uv'].sum()

    # 计算每个文件夹的CTR
    for folder in folder_stats:
        if folder_stats[folder]['uv'] > 0:
            folder_stats[folder]['ctr'] = folder_stats[folder]['click_uv'] / folder_stats[folder]['uv']
        else:
            folder_stats[folder]['ctr'] = 0

    return folder_stats

all_stats = {}

for root, dirs, files in os.walk(base_path):
    if 'grounding_output' in dirs:
        grounding_path = os.path.join(root, 'grounding_output')
        folder_name = os.path.basename(root)
        stats = process_grounding_folder(grounding_path)
        all_stats[folder_name] = stats

# 创建一个Excel工作簿来保存结果
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.title = "Image Distribution Stats"

# 写入表头
headers = ["Folder", "Subfolder", "Count", "UV", "Click UV", "CTR"]
for col, header in enumerate(headers, start=1):
    sheet.cell(row=1, column=col, value=header)

row = 2  # 从第二行开始写入数据

# 输出统计结果
for folder, stats in all_stats.items():
    print(f"\n统计结果 for {folder}:")
    for subfolder, data in stats.items():
        if data['count'] > 0:
            ctr = data['click_uv'] / data['uv'] if data['uv'] > 0 else 0
            print(f"  {subfolder}: 图片数量 = {data['count']}, UV = {data['uv']}, Click UV = {data['click_uv']}, CTR = {ctr:.4f}")

# 定义颜色映射
color_map = {
    'price': '#FF6B6B',  # 柔和的红色
    'txt': '#4ECDC4',    # 青绿色
    'scene': '#7986CB',  # 淡紫色
    'white': '#FFD93D'   # 明亮的黄色
}

# 绘制每个grounding_output文件夹的饼图并保存数据
for folder, stats in all_stats.items():
    counts = [data['count'] for data in stats.values() if data['count'] > 0]
    labels = [subfolder for subfolder, data in stats.items() if data['count'] > 0]
    ctrs = [data['ctr'] for data in stats.values() if data['count'] > 0]
    colors = [color_map.get(label, 'gray') for label in labels]  # 使用颜色映射，如果没有指定则默认为灰色
    
    if counts:  # 只有当有数据时才绘图
        plt.figure(figsize=(12, 9))
        wedges, texts, autotexts = plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
        
        # 添加数量和CTR标签
        for i, (autotext, ctr) in enumerate(zip(autotexts, ctrs)):
            autotext.set_text(f'{autotext.get_text()}\n({counts[i]})\nCTR: {ctr:.4f}')
        
        # 在饼图名中添加筛选条件的值
        plt.title(f"Image Distribution in {folder} ({', '.join([str(layer) for layer in filter_layers])})")
        plt.axis('equal')
        
        # 添加图例
        plt.legend(wedges, [f"{label} ({count}, CTR: {ctr:.4f})" for label, count, ctr in zip(labels, counts, ctrs)],
                   title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
        
        # 在输出路径中添加筛选条件的值
        output_chart_path = os.path.join(base_path, f'{filter_layers}_{folder}_distribution.png')
        plt.savefig(output_chart_path, bbox_inches='tight')
        plt.close()
        print(f"饼图已保存至: {output_chart_path}")

        # 将图片插入到Excel中
        img = XLImage(output_chart_path)
        sheet.add_image(img, f'H{row}')

        # 写入数据到Excel
        for subfolder, data in stats.items():
            if data['count'] > 0:
                sheet.cell(row=row, column=1, value=folder)
                sheet.cell(row=row, column=2, value=subfolder)
                sheet.cell(row=row, column=3, value=data['count'])
                sheet.cell(row=row, column=4, value=data['uv'])
                sheet.cell(row=row, column=5, value=data['click_uv'])
                sheet.cell(row=row, column=6, value=data['ctr'])
                row += 1

        row += 15  # 为下一个文件夹的数据和图表留出空间

# 计算总体统计
total_stats = {
    'price': {'count': 0, 'uv': 0, 'click_uv': 0},
    'txt': {'count': 0, 'uv': 0, 'click_uv': 0},
    'scene': {'count': 0, 'uv': 0, 'click_uv': 0},
    'white': {'count': 0, 'uv': 0, 'click_uv': 0}
}

for folder_stats in all_stats.values():
    for subfolder, data in folder_stats.items():
        total_stats[subfolder]['count'] += data['count']
        total_stats[subfolder]['uv'] += data['uv']
        total_stats[subfolder]['click_uv'] += data['click_uv']

# 计算总体CTR
for subfolder in total_stats:
    if total_stats[subfolder]['uv'] > 0:
        total_stats[subfolder]['ctr'] = total_stats[subfolder]['click_uv'] / total_stats[subfolder]['uv']
    else:
        total_stats[subfolder]['ctr'] = 0

# 打印总体统计结果
print("\n总体统计结果:")
for subfolder, data in total_stats.items():
    if data['count'] > 0:
        print(f"  {subfolder}: 总图片数量 = {data['count']}, 总UV = {data['uv']}, 总Click UV = {data['click_uv']}, 总CTR = {data['ctr']:.4f}")

# 绘制总体分布饼图
total_counts = [data['count'] for data in total_stats.values() if data['count'] > 0]
total_labels = [subfolder for subfolder, data in total_stats.items() if data['count'] > 0]
total_ctrs = [data['click_uv'] / data['uv'] if data['uv'] > 0 else 0 for data in total_stats.values() if data['count'] > 0]
total_colors = [color_map.get(label, 'gray') for label in total_labels]

if total_counts:
    plt.figure(figsize=(12, 9))
    wedges, texts, autotexts = plt.pie(total_counts, labels=total_labels, autopct='%1.1f%%', startangle=90, colors=total_colors)
    
    # 添加数量和CTR标签
    for i, (autotext, ctr) in enumerate(zip(autotexts, total_ctrs)):
        autotext.set_text(f'{autotext.get_text()}\n({total_counts[i]})\nCTR: {ctr:.4f}')
    
    plt.title("Overall Image Distribution")
    plt.axis('equal')
    
    # 添加图例
    plt.legend(wedges, [f"{label} ({count}, CTR: {ctr:.4f})" for label, count, ctr in zip(total_labels, total_counts, total_ctrs)],
               title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    
    output_chart_path = os.path.join(base_path, f'{filter_layers}_overall_distribution.png')
    plt.savefig(output_chart_path, bbox_inches='tight')
    plt.close()
    print(f"总体分布饼图已保存至: {output_chart_path}")

    # 将总体分布图片插入到Excel中
    img = XLImage(output_chart_path)
    sheet.add_image(img, f'H{row}')

    # 写入总体数据到Excel
    sheet.cell(row=row, column=1, value="Overall")
    for subfolder, data in total_stats.items():
        if data['count'] > 0:
            sheet.cell(row=row, column=2, value=subfolder)
            sheet.cell(row=row, column=3, value=data['count'])
            sheet.cell(row=row, column=4, value=data['uv'])
            sheet.cell(row=row, column=5, value=data['click_uv'])
            sheet.cell(row=row, column=6, value=data['click_uv'] / data['uv'] if data['uv'] > 0 else 0)
            row += 1

# 在Excel表名中添加筛选条件的值
sheet.title = f"Image Distribution Stats ({', '.join([str(layer) for layer in filter_layers])})"

# 保存Excel文件
excel_output_path = os.path.join(base_path, f'{filter_layers}_image_distribution_stats.xlsx')
workbook.save(excel_output_path)
print(f"统计结果已保存至Excel文件: {excel_output_path}")




C:\Users\heyunshen\AppData\Local\Temp\ipykernel_14644\1718970685.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['matching_part'] = filtered_df['img_url'].apply(extract_matching_part)



统计结果 for 12066:
  scene: 图片数量 = 3, UV = 119, Click UV = 9, CTR = 0.0756

统计结果 for 6908:
  price: 图片数量 = 2, UV = 185646, Click UV = 7092, CTR = 0.0382
  txt: 图片数量 = 21, UV = 266724, Click UV = 7553, CTR = 0.0283
  scene: 图片数量 = 101, UV = 2417340, Click UV = 76598, CTR = 0.0317
  white: 图片数量 = 20, UV = 564649, Click UV = 16613, CTR = 0.0294

统计结果 for 6909:
  txt: 图片数量 = 199, UV = 1029709, Click UV = 34086, CTR = 0.0331
  scene: 图片数量 = 54, UV = 449102, Click UV = 13960, CTR = 0.0311
  white: 图片数量 = 17, UV = 295162, Click UV = 9235, CTR = 0.0313

统计结果 for 6910:
  price: 图片数量 = 4, UV = 17699, Click UV = 485, CTR = 0.0274
  txt: 图片数量 = 132, UV = 753243, Click UV = 24031, CTR = 0.0319
  scene: 图片数量 = 88, UV = 1242612, Click UV = 44175, CTR = 0.0356
  white: 图片数量 = 101, UV = 259364, Click UV = 8371, CTR = 0.0323

统计结果 for 6911:
  price: 图片数量 = 8, UV = 64490, Click UV = 2814, CTR = 0.0436
  txt: 图片数量 = 205, UV = 2779763, Click UV = 98562, CTR = 0.0355
  scene: 图片数量 = 93, UV = 2339507, Click UV

c:\ProgramData\miniconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
